# [Building a Vision Transformer Model From Scratch](https://medium.com/correll-lab/building-a-vision-transformer-model-from-scratch-a3054f707cc6)

## Import Libraries and Modules

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.optim import Adam
from torchvision.datasets.mnist import MNIST
from torch.utils.data import DataLoader
import numpy as np

우리는 PyTorch를 사용하여 Vision Transformer를 구축할 예정이므로,  <br>
이 튜토리얼에서 사용할 PyTorch와 기타 라이브러리들을 임포트해야 합니다.

In [3]:
import torch
import torch.nn as nn

또한 입력 이미지를 리사이즈하고 텐서로 변환하기 위해 `torchvision.transforms`를 <br>
임포트해야 합니다. 입력 이미지를 리사이즈하는 것은 선택 사항입니다. <br>
다만, 이미지 크기가 패치 크기로 나누어떨어지도록만 하면 됩니다.

In [4]:
import torchvision.transforms as T

우리는 옵티마이저로 Adam을 사용할 예정이므로, `torch.optim`에서 <br>
이를 임포트해야 합니다.

In [6]:
from torch.optim import Adam

이번 튜토리얼에서 사용할 MNIST 데이터셋을 `torchvision`에서 임포트할 예정입니다. <br>
또한 데이터를 불러오는 데 도움이 되는 PyTorch의 `DataLoader`를 사용할 것이므로, <br>
그것도 함께 임포트해야 합니다.

In [8]:
from torchvision.datasets.mnist import MNIST
from torch.utils.data import DataLoader

마지막으로, 위치 인코딩을 생성할 때 사인(sin)과 코사인(cosine) 연산을<br>
수행하기 위해 사용할 numpy를 임포트해야 합니다.

In [10]:
import numpy as np

## Patch Embeddings

In [11]:
class PatchEmbedding(nn.Module):
  def __init__(self, d_model, img_size, patch_size, n_channels):
    super().__init__()

    self.d_model = d_model # Dimensionality of Model
    self.img_size = img_size # Image Size
    self.patch_size = patch_size # Patch Size
    self.n_channels = n_channels # Number of Channels

    self.linear_project = nn.Conv2d(self.n_channels, self.d_model, kernel_size=self.patch_size, stride=self.patch_size)

  # B: Batch Size
  # C: Image Channels
  # H: Image Height
  # W: Image Width
  # P_col: Patch Column
  # P_row: Patch Row
  def forward(self, x):
    x = self.linear_project(x) # (B, C, H, W) -> (B, d_model, P_col, P_row)

    x = x.flatten(2) # (B, d_model, P_col, P_row) -> (B, d_model, P)

    x = x.transpose(1, 2) # (B, d_model, P) -> (B, P, d_model)
    
    return x

비전 트랜스포머를 만드는 첫 번째 단계는 입력 이미지를 패치로 분할하고, <br>
이 패치들의 선형 임베딩 시퀀스를 생성하는 것입니다. <br>
이는 PyTorch의 Conv2d 메서드를 사용하여 구현할 수 있습니다. <br>
<br>
Conv2d 메서드는 입력 이미지를 받아 패치로 나눈 뒤, 모델의 너비와 동일한 크기의<br>
선형 투영을 제공합니다. kernel_size와 stride를 패치 크기로 설정함으로써, <br>
패치들이 올바른 크기를 가지며 서로 겹치지 않도록 보장할 수 있습니다.